# Importing all libraries needed for program

In [0]:
import matplotlib.pyplot as plt
import os
import shutil
from keras import layers
from keras import models
from keras.models import load_model
from keras import optimizers
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

from statistics import mean

from PIL import Image

Using TensorFlow backend.


In [0]:
!pip install PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1EgZuR10bXkNxK3hg1ppfz_T2aRuUxXwC'})
download.GetContentFile('dataset.tar')

# model = drive.Create({'id': ''})
# model.GetContentFile('model.h5')

In [0]:
!tar -xvf dataset.tar

In [0]:
def dataset_cleaner(root):
    for path, sub, files in os.walk(root):
        if '.ipynb_checkpoints' in path:
            shutil.rmtree(path)
        for name in files:
            if name[0] == '.':
                os.remove(os.path.join(path, name))

In [0]:
!ls -a

dataset_cleaner(os.getcwd())

In [0]:
os.chdir('data/test')

!ls -a

dataset_cleaner(os.getcwd())

os.chdir('../..')

In [0]:
os.chdir('data/train')

!ls -a
dataset_cleaner(os.getcwd())
    
os.chdir('../..')

In [0]:
os.chdir('data/validation')

!ls -a
dataset_cleaner(os.getcwd())

os.chdir('../..')

# Finds the average image width and height

In [0]:
def width_height_of_images(root='../static/data'):
    image_width = []
    image_height = []
    for path, subdirs, files in os.walk(root):
        for name in files:
            try:
                image = Image.open(os.path.join(path, name))
                # print(image.format)
                # print(image.size)
                width, height = image.size
                image_width.append(width)
                image_height.append(height)
                image.close()
            except OSError:
                pass
            # except UnboundLocalError:
            #     print(path, name)
            # image.close()
    return int(mean(image_width)), int(mean(image_height))

# Main class for Image Training

In [0]:
# dogs = DogBreedTraining('data')
# dogs.directory_init()
# dogs.image_data_gen(20)

# print(dogs.train_generator.class_indices.keys())

# dogs.dogs_breed_modeling()
# dogs.results()

In [0]:
class DogBreedTraining:
    def __init__(self, base_dir='static/data'):
#         plaidml.keras.install_backend()
        self.avg_width, self.avg_height = width_height_of_images(base_dir)
        self.conv_base = VGG16(weights='imagenet',
                               include_top=False,
                               input_shape=(256, 256, 3))

        base_dir = base_dir

        self.train_dir = os.path.join(base_dir, 'train')

        self.validation_dir = os.path.join(base_dir, 'train')

        self.test_dir = os.path.join(base_dir, 'test')

        self.bulldog = 'bulldog'
        self.german_shepherd = 'german_shepherd'
        self.golden_retriever = 'golden_retriever'
        self.husky = 'husky'
        self.poodle = 'poodle'
        self.breeds_of_dogs = [self.bulldog, 
                               self.german_shepherd, 
                               self.golden_retriever, 
                               self.husky, 
                               self.poodle]

        self.train_bulldog_dir = None
        self.train_german_shepherd_dir = None
        self.train_golden_retriever_dir = None
        self.train_husky_dir = None
        self.train_poodle_dir = None

        self.validation_bulldog_dir = None
        self.validation_german_shepherd_dir = None
        self.validation_golden_retriever_dir = None
        self.validation_husky_dir = None
        self.validation_poodle_dir = None

        self.test_bulldog_dir = None
        self.test_german_shepherd_dir = None
        self.test_golden_retriever_dir = None
        self.test_husky_dir = None
        self.test_poodle_dir = None

        self.train_generator = None
        self.validation_generator = None
        self.test_datagen = None

        self.history = None

    def directory_init(self):
        self.train_bulldog_dir = os.path.join(self.train_dir, self.bulldog)
        self.train_german_shepherd_dir = os.path.join(self.train_dir, self.german_shepherd)
        self.train_golden_retriever_dir = os.path.join(self.train_dir, self.golden_retriever)
        self.train_husky_dir = os.path.join(self.train_dir, self.husky)
        self.train_poodle_dir = os.path.join(self.train_dir, self.poodle)

        self.validation_bulldog_dir = os.path.join(self.validation_dir, self.bulldog)
        self.validation_german_shepherd_dir = os.path.join(self.validation_dir, self.german_shepherd)
        self.validation_golden_retriever_dir = os.path.join(self.validation_dir, self.golden_retriever)
        self.validation_husky_dir = os.path.join(self.validation_dir, self.husky)
        self.validation_poodle_dir = os.path.join(self.validation_dir, self.poodle)

        self.test_bulldog_dir = os.path.join(self.test_dir, self.bulldog)
        self.test_german_shepherd_dir = os.path.join(self.test_dir, self.german_shepherd)
        self.test_golden_retriever_dir = os.path.join(self.test_dir, self.golden_retriever)
        self.test_husky_dir = os.path.join(self.test_dir, self.husky)
        self.test_poodle_dir = os.path.join(self.test_dir, self.poodle)

    def image_data_gen(self, batch_size):
        train_datagen = ImageDataGenerator(
            rescale=1. / 255,
            rotation_range=40, 
            width_shift_range=0.2, 
            height_shift_range=0.2, 
            shear_range=0.2, 
            zoom_range=0.2, 
            horizontal_flip=True, 
            fill_mode='nearest')
        
        validation_datagen = ImageDataGenerator(rescale=1. / 255)
        
        self.test_datagen = ImageDataGenerator(rescale=1. / 255)

        self.train_generator = train_datagen.flow_from_directory(
            self.train_dir,
            target_size=(256, 256),
            batch_size=batch_size,
            class_mode='categorical'
        )

        self.validation_generator = validation_datagen.flow_from_directory(
            self.validation_dir,
            target_size=(256, 256),
            batch_size=batch_size,
            class_mode='categorical'
        )

    def dogs_breed_modeling(self):
        model = models.Sequential()
        model.add(self.conv_base)
        model.summary()
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))
        
        print('This is the number of trainable weights ' 'before freezing the conv base:', len(model.trainable_weights))
        self.conv_base.trainable = False
        print('This is the number of trainable weights ' 'after freezing the conv base:', len(model.trainable_weights))
        
        model.summary()

        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(),
                      metrics=['accuracy'])

        self.history = model.fit_generator(
            self.train_generator,
            steps_per_epoch=100,
            epochs=30,
            validation_data=self.validation_generator,
            validation_steps=50
        )

        model.save('dogs_breed_1.h5')
    
    def dogs_breed_modeling_part_2(self):
        model = models.Sequential()
        model.add(self.conv_base)
        model.summary()
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))
        
        print('This is the number of trainable weights ' 'before freezing the conv base:', len(model.trainable_weights))
        self.conv_base.trainable = False
        print('This is the number of trainable weights ' 'after freezing the conv base:', len(model.trainable_weights))
        
        model.summary()

        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(),
                      metrics=['accuracy'])

        self.history = model.fit_generator(
            self.train_generator,
            steps_per_epoch=200,
            epochs=100,
            validation_data=self.validation_generator,
            validation_steps=75
        )

        model.save('dogs_breed_2.h5')
        
    def dogs_breed_modeling_part_3(self):
        model = models.Sequential()
        model.add(self.conv_base)
        model.summary()
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))
        
        print('This is the number of trainable weights ' 'before freezing the conv base:', len(model.trainable_weights))
        self.conv_base.trainable = False
        print('This is the number of trainable weights ' 'after freezing the conv base:', len(model.trainable_weights))
        
        model.summary()

        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(lr=0.000125, epsilon=None, 
                                                decay=0.1, amsgrad=False),
                      metrics=['accuracy'])

        self.history = model.fit_generator(
            self.train_generator,
            steps_per_epoch=20,
            epochs=50,
            validation_data=self.validation_generator,
            validation_steps=50
        )

        model.save('dogs_breed_7.h5')
    
    def dogs_breed_modeling_part_4(self):
        model = models.Sequential()
        model.add(self.conv_base)
        model.summary()
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(5, activation='softmax'))
        
        print('This is the number of trainable weights ' 'before freezing the conv base:', len(model.trainable_weights))
        self.conv_base.trainable = False
        print('This is the number of trainable weights ' 'after freezing the conv base:', len(model.trainable_weights))
        
        model.summary()

        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(lr=0.00002, beta_1=0.9, 
                                                beta_2=0.999, epsilon=None, 
                                                decay=0.5, amsgrad=False),
                      metrics=['accuracy'])

        self.history = model.fit_generator(
            self.train_generator,
            steps_per_epoch=500,
            epochs=250,
            validation_data=self.validation_generator,
            validation_steps=100
        )

        model.save('dogs_breed_4.h5')
        
    def model_evaluate(self):
        model = load_model('dogs_breed_4 (1).h5')
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        # print(model.predict('static/bulldog/1. Regular/91.Regular.png'))
        test_dir = "static/data/test"
        test_datagen = ImageDataGenerator(rescale=1./255)
        test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(256, 256),
                                                  batch_size=20,
                                                  class_mode='categorical')

        # predict the result

        test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
        print(test_loss, test_acc)

    def results(self):
        acc = self.history.history['acc']
        val_acc = self.history.history['val_acc']
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']

        epochs = range(1, len(acc) + 1)

        plt.plot(epochs, acc, 'bo', label='Training acc')
        plt.plot(epochs, val_acc, 'b', label='Validation acc')
        plt.title('Training and validation accuracy')
        plt.legend()

        plt.figure()

        plt.plot(epochs, loss, 'bo', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.legend()

        plt.show()


In [0]:
# dogs_part_2 = DogBreedTraining('data')
# dogs_part_2.directory_init()
# dogs_part_2.image_data_gen(50)

# print(dogs_part_2.train_generator.class_indices.keys())
# dogs_part_2.dogs_breed_modeling_part_2()
# dogs_part_2.results()

# # files.download('dogs_breed_2.h5')

In [0]:
# dogs_part_4 = DogBreedTraining('data')
# dogs_part_4.directory_init()
# dogs_part_4.image_data_gen(100)

# print(dogs_part_4.train_generator.class_indices.keys())
# dogs_part_4.dogs_breed_modeling_part_4()
# dogs_part_4.results()

# files.download('dogs_breed_4.h5')

In [0]:
dogs_part_3 = DogBreedTraining('data')
dogs_part_3.directory_init()
dogs_part_3.image_data_gen(50)

print(dogs_part_3.train_generator.class_indices.keys())
dogs_part_3.dogs_breed_modeling_part_3()
dogs_part_3.results()

# files.download('dogs_breed_3.h5')